In [1]:
!pip install transformers accelerate #  모델의 GPU/TPU 가속화와 분산 학습을 도와주는 라이브러리

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Bllossom/llama-3.2-Korean-Bllossom-3B")
model = AutoModelForCausalLM.from_pretrained("Bllossom/llama-3.2-Korean-Bllossom-3B")

model.config.pad_token_id = model.config.eos_token_id

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

In [3]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
save_path = "/content/drive/MyDrive/yaife/model_checkpoint"

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('/content/drive/MyDrive/yaife/model_checkpoint/tokenizer_config.json',
 '/content/drive/MyDrive/yaife/model_checkpoint/special_tokens_map.json',
 '/content/drive/MyDrive/yaife/model_checkpoint/tokenizer.json')

In [5]:
# 모델 로드
load_path = "/content/drive/MyDrive/yaife/model_checkpoint"

tokenizer = AutoTokenizer.from_pretrained(load_path)
model = AutoModelForCausalLM.from_pretrained(load_path)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
tokenizer.pad_token = tokenizer.eos_token

In [7]:
# test 1- with 유해요소 template
input_text = """
유해 요소: ['구제불능'].
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경하세요:
기존 문장: 그런 인간은 정말 구제불능이다.
수정된 문장:"""

# Tokenizer로 입력 데이터 준비
inputs = tokenizer(
    input_text,
    return_tensors="pt",
    padding=True,
    truncation=True,
    return_attention_mask=True
)

# Model generation with attention mask and pad token id 설정
initial_output = model.generate(
    inputs['input_ids'],  # 입력 ID
    attention_mask=inputs['attention_mask'],  # attention mask 추가
    max_new_tokens=250,
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id  # pad_token_id 설정
)

initial_result = tokenizer.decode(initial_output[0], skip_special_tokens=True)
print("Initial Result:", initial_result)


Initial Result: 
유해 요소: ['구제불능'].
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경하세요:
기존 문장: 그런 인간은 정말 구제불능이다.
수정된 문장: 그런 인간은 정말 도움을 받지 못한다.
기존 문장: 그는 이 일을 할 수 없다는 것을 알게 되었다.
수정된 문장: 그는 이 일을 할 수 없다는 것을 알게 되었다.
기존 문장: 그런 인간은 정말 구제불능이다.
수정된 문장: 그런 인간은 정말 도움을 받지 못한다.
기존 문장: 그는 이 일을 할 수 없다는 것을 알게 되었다.
수정된 문장: 그는 이 일을 할 수 없다는 것을 알게 되었다. 

결론: 문장에서 '구제불능'이라는 표현을 '도움을 받지 못한다'로 변경하면 긍정적인 의미를 더할 수 있습니다. 이는 문장의 내용을 변경하지 않으면서도 표현의 양해를 얻는 방법입니다. 따라서, '구제불능'은 부정적인 표현으로, '도움을 받지 못한다'는 표현은 긍정적인 의미를 더하는 방법입니다. 이와 같이 문장을 수정하면 긍정적인 표현으로 전환할 수 있습니다. 

이와 같은 방법을 통해 문장의 표현을 양해를 얻


In [8]:
# test2- no 유해요소 template
input_text_without_template = """
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경하세요:
기존 문장: 그런 인간은 정말 구제불능이다.
수정된 문장:"""

without_output = model.generate(
    tokenizer(input_text_without_template, return_tensors="pt", padding=True, truncation=True, return_attention_mask=True)['input_ids'],
    max_new_tokens=250,
    temperature=0.7,
    top_k=50,
    top_p=0.9
)

without_result = tokenizer.decode(without_output[0], skip_special_tokens=True)
print("without_result:", without_result)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


without_result: 
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경하세요:
기존 문장: 그런 인간은 정말 구제불능이다.
수정된 문장: 그런 인간은 정말 도움이 필요하다.

이 문장을 통해 사용된 변경은 다음과 같습니다:
- '구제불능' -> '도움이 필요하다': 이 문장은 '구제불능'이라는 단어가 부정적인 의미를 가지고 있지만, '도움이 필요하다'라는 표현은 긍정적인 의미를 가지고 있어 문장을 긍정적으로 변경하는 데 사용됩니다. 또한 '구제불능'이라는 단어는 특정 상황에 의존하는 사람을 말하는 데 사용되지만, '도움이 필요하다'는 모든 사람에게 도움이 필요한 상황을 나타내는 표현입니다. 따라서 '도움이 필요하다'는 더 넓은 범위의 의미를 가지고 있어 문장을 긍정적으로 변경하는 데 사용됩니다. 또한 '구제불능'이라는 단어는 '구제'라는 단어가 부정적인 의미를 가지고 있지만, '도움이 필요하다'는 '도움이 필요하다'라는 단어가 긍정적인 의미를 가지고 있어 문장을 긍정적으로 변경하는 데 사용됩니다. 따라서 '도움이 필요하다'는 더 넓은 범


In [9]:
# test 3- 더 길고 복잡한 문장으로 테스트 예제 1번
input_text_test3 = """
유해 요소: ['무식', '아무 쓸모도 없는', '시간 낭비', '피해', '구제불능', '대체 불가능한 문제'].
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경하세요:
기존 문장: 그 사람은 정말 무식하고 아무 쓸모도 없는 사람이다. 대화를 해보면 인내심이 바닥나고 그와 시간을 보내는 것이 시간 낭비라고 느껴질 정도다. 특히 그의 행동은 항상 다른 사람들에게 피해를 주며, 그의 성격은 구제불능일 뿐 아니라 대체 불가능한 문제를 안고 있다. 그가 이 사회에서 어울릴 수 있을지 의문이다.
수정된 문장:"""

test3_output = model.generate(
    tokenizer(input_text_test3, return_tensors="pt", padding=True, truncation=True, return_attention_mask=True)['input_ids'],
    max_new_tokens=250,
    temperature=0.5,
    top_k=50,
    top_p=0.9,
    no_repeat_ngram_size=3,
    pad_token_id=tokenizer.eos_token_id
)

test3_result = tokenizer.decode(test3_output[0], skip_special_tokens=True)
print("test3_Result:", test3_result)



test3_Result: 
유해 요소: ['무식', '아무 쓸모도 없는', '시간 낭비', '피해', '구제불능', '대체 불가능한 문제'].
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경하세요:
기존 문장: 그 사람은 정말 무식하고 아무 쓸모도 없는 사람이다. 대화를 해보면 인내심이 바닥나고 그와 시간을 보내는 것이 시간 낭비라고 느껴질 정도다. 특히 그의 행동은 항상 다른 사람들에게 피해를 주며, 그의 성격은 구제불능일 뿐 아니라 대체 불가능한 문제를 안고 있다. 그가 이 사회에서 어울릴 수 있을지 의문이다.
수정된 문장: 그는 정말 무지하고 쓸데없는 사람일 수 있다. 대화를 통해 그의 인내가 부족하다는 것을 알 수 있다, 그리고 시간을 함께 보내는 것은 시간을 낭만스럽게 보는 것일 수 있지만, 그의 행동으로 인해 다른 사람들을 상처를 주고 있다, 그의성격은 도움이 되지 않는 문제일 뿐이다. 그가 사회에서 잘 어울리는지 의심할 만하다.

유해한 요소는 다음과 같습니다:
- 무식: 무지로 대체
- 아무 쓴모도: 쓸쓸한 사람일수도
- 시간 낙비: 시간 낯설게 보이게 하거나 낭漫하게 보이도록
- 피해: 상처 주어
- 구제 불능: 도웘이 되지 않아
- 대체불가능한: 도움을 받을 수 없다는 의미로 대신 사용
- 피해: 상해
- 무지: 무지는 대체 가능

이러한 표현들을 수정하여 긍 정적인 문장을 만들 수 있습니다. 또한, 유해 요소를 제거하여 문장이 더 긍


In [10]:
# Self-reflection
reflection_input = f"""
생성된 문장은 다음과 같습니다:
"{test3_result}"
이 문장은 원래 문장의 의미를 보존하며 유해요소를 완화하는 표현으로 변경되었나?
문맥에 맞지 않다면, 올바른 표현으로 다시 수정해줘. 완화되었고 문맥과 비슷하다면 생성을 멈춰줘.
"""

reflection_output = model.generate(
    tokenizer(reflection_input, return_tensors="pt", padding=True, truncation=True, return_attention_mask=True)['input_ids'],
    max_new_tokens=250,
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    eos_token_id=tokenizer.eos_token_id
)

final_result = tokenizer.decode(reflection_output[0], skip_special_tokens=True)
print("Final Result after Reflection:", final_result)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Final Result after Reflection: 
생성된 문장은 다음과 같습니다:
"
유해 요소: ['무식', '아무 쓸모도 없는', '시간 낭비', '피해', '구제불능', '대체 불가능한 문제'].
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경하세요:
기존 문장: 그 사람은 정말 무식하고 아무 쓸모도 없는 사람이다. 대화를 해보면 인내심이 바닥나고 그와 시간을 보내는 것이 시간 낭비라고 느껴질 정도다. 특히 그의 행동은 항상 다른 사람들에게 피해를 주며, 그의 성격은 구제불능일 뿐 아니라 대체 불가능한 문제를 안고 있다. 그가 이 사회에서 어울릴 수 있을지 의문이다.
수정된 문장: 그는 정말 무지하고 쓸데없는 사람일 수 있다. 대화를 통해 그의 인내가 부족하다는 것을 알 수 있다, 그리고 시간을 함께 보내는 것은 시간을 낭만스럽게 보는 것일 수 있지만, 그의 행동으로 인해 다른 사람들을 상처를 주고 있다, 그의성격은 도움이 되지 않는 문제일 뿐이다. 그가 사회에서 잘 어울리는지 의심할 만하다.

유해한 요소는 다음과 같습니다:
- 무식: 무지로 대체
- 아무 쓴모도: 쓸쓸한 사람일수도
- 시간 낙비: 시간 낯설게 보이게 하거나 낭漫하게 보이도록
- 피해: 상처 주어
- 구제 불능: 도웘이 되지 않아
- 대체불가능한: 도움을 받을 수 없다는 의미로 대신 사용
- 피해: 상해
- 무지: 무지는 대체 가능

이러한 표현들을 수정하여 긍 정적인 문장을 만들 수 있습니다. 또한, 유해 요소를 제거하여 문장이 더 긍"
이 문장은 원래 문장의 의미를 보존하며 유해요소를 완화하는 표현으로 변경되었나?
문맥에 맞지 않다면, 올바른 표현으로 다시 수정해줘. 완화되었고 문맥과 비슷하다면 생성을 멈춰줘.
문장이 원래 문장의 의미를 보존하고 유해요소를 완화한 것이 아니라면, 다시 수정해 주세요.
문장이 완화되었고 문맥이 잘 맞는지 확인해 주세요.
문장이 완화되었고 문맥이 잘 맞다면 생성을 마무리해 주세요.
문장이 완화되었고 문맥이 잘 맞다면 생성을 마무리해 주세요

In [11]:
# 유사도를 이용한 재생성 여부 판단하기
from difflib import SequenceMatcher

def calculate_similarity(a, b):
    """두 텍스트 간 유사도를 계산"""
    return SequenceMatcher(None, a, b).ratio()

def is_satisfactory(output):
    """모델의 출력이 적합한지 판단"""
    return "완화된 표현으로 변경되었습니다" in output or "완료" in output

prev_result = None
MAX_ITERATIONS = 3

for i in range(MAX_ITERATIONS):
    reflection_input = f"""
    이전 결과: "{prev_result if prev_result else "초기 입력"}"
    문맥에 맞지 않거나 의미가 왜곡된 경우, 올바른 문장으로 수정해줘. 하지만 문맥에 맞거나 바꿀 필요가 없다면 새로운 생성을 멈춰줘.
    """

    inputs = tokenizer(input_text_test3, return_tensors="pt", padding=True, truncation=True, return_attention_mask=True)
    output = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_new_tokens=250,
        temperature=0.7,
        top_k=50,
        top_p=0.9,
        no_repeat_ngram_size=3,
        pad_token_id=tokenizer.eos_token_id
    )

    new_result = tokenizer.decode(output[0], skip_special_tokens=True).strip()

    # 적합한 결과인지 확인
    if is_satisfactory(new_result):
        print(f"Iteration {i+1}: Satisfactory result found, stopping loop.")
        print("Final Result:", new_result)
        break

    # 유사도가 높은 경우 중단 (0.8이상)
    if prev_result and calculate_similarity(prev_result, new_result) > 0.8:
        print(f"Iteration {i+1}: No significant improvement, stopping loop.")
        print("Final Result:", new_result)
        break


    prev_result = new_result
    print(f"Iteration {i+1} Result:", new_result)


Iteration 1 Result: 유해 요소: ['무식', '아무 쓸모도 없는', '시간 낭비', '피해', '구제불능', '대체 불가능한 문제'].
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경하세요:
기존 문장: 그 사람은 정말 무식하고 아무 쓸모도 없는 사람이다. 대화를 해보면 인내심이 바닥나고 그와 시간을 보내는 것이 시간 낭비라고 느껴질 정도다. 특히 그의 행동은 항상 다른 사람들에게 피해를 주며, 그의 성격은 구제불능일 뿐 아니라 대체 불가능한 문제를 안고 있다. 그가 이 사회에서 어울릴 수 있을지 의문이다.
수정된 문장: 그는 정말 무지한 사람이라고 생각한다. 대화하면 인내가 소모되고, 시간을 함께 보는 것은 시간 납득하지 않다는 느낌을 받는다. 특히 그가 행동하는 방식은 다른 사람에게 피해를 가며, 그는 성격적으로 구제 불가하고 대체 가능한 문제를抱고 있다.
이제 주어진 문장을 더 긍성적으로 변경해보겠습니다.
기존문장: 
- "그 사람은 정말무식하고어떤 쓸쾌도없는 사람이다."
- "대화하면인내심바닥나서그와시간을보내는것은시간낭비라는 느낌이온다."
- "(그의)행동은항상다른사람들에게피해를주며,(그의성격)구제 불가능하고대체불가능한문제를안고있다."

수정문장:
- "이 사람은 정말지식이 부족하고어려운 상황에 직면한 사람이다." 
- "(이 사람과 대화할 때)인내가소모되고,(이 시간을 지속할 수 있는지)시간 납得지 않는다
Iteration 2 Result: 유해 요소: ['무식', '아무 쓸모도 없는', '시간 낭비', '피해', '구제불능', '대체 불가능한 문제'].
주어진 문장에서 유해한 표현을 제거하고 긍정적으로 변경하세요:
기존 문장: 그 사람은 정말 무식하고 아무 쓸모도 없는 사람이다. 대화를 해보면 인내심이 바닥나고 그와 시간을 보내는 것이 시간 낭비라고 느껴질 정도다. 특히 그의 행동은 항상 다른 사람들에게 피해를 주며, 그의 성격은 구제불능일 뿐 아니라 대체 불가능한 문제를 안고 있다. 그가 이 사회에서 어울릴 수 있을지 의문이